# FINAL REPORT ---- Yulin Lei & Wenli Shi
### Chosen paper: Latent Dirichlet Allocation by David M. Blei, Andrew Y. Ng and Michael I. Jordan

## 1. Background Introduction

### A. Question Interested

Due to the explosion of data and documents, the algorithm for modeling text corpora is urgent to be created because of the difficulty for managing documents. Before this paper, several models have be proposed by statistician and computer scientist, like information retrieval (IR) (Baeza-Yates and Ribeiro-Neto, 1999), latent semantic indexing (LSI) (Deerwester et al., 1990) and probabilistic LSI (pLSI) model (Hofmann 1999). From the current paper, the authors construct a Bayesian hierarchical model to describe the process of generating the documents in a corpora called Latent Dirichlet Allocation model (LDA). 

### B. Basic Assumptions, Thoughts and Procedure

As those previous papers, this LDA model also treats the corpora as a three-level system that corpora contains documents contains bunch of words and considers words as the basic unit. However, this LDA model builds up a generating process for the words as likelihood of the Bayesian hierarchical model and assigned some priors for this process. This LDA model is constructed based on the assumption of exchangeability. Also, because of this assumption, this model ignores the relation between the adjacent words and thereby could classify them into different topics. An improvement for this is considering adding Markov Property onto the likelihood to modify the assumption of exchangeability. 

One feature of this model is that it adds a latent variables as "topics" in the model. Therefore, the process of generating words is based on the topics. While each word is associated with one topic, one document is a mixture of topics by the words in that document. To infer the interested parameteres in the model, the authors adopts variation inference and EM algorithm because of the difficulty of inference directly.

## 2. Algorithm 

### A. Models

#### (1). Notation

To state the detailed algorithm formally, the authors define the following terms:
- A _word_ is the basic unit of discrete data, defined to be an item from a vocabulary indexed by $\{1, \cdots , V \}$. We represent words using unit-basis vectors that have a single component equal to one and all other components equal to zero. Thus, using superscripts to denote components, the _v_th word in the vocabulary is represented by a V-vector $w$ such that $w^v = 1$ and $w^u = 0$ for $u\neq v$.
- A _document_ is a sequence of $N$ words denoted by $\textbf{w} = (w_1,w_2,... ,w_N)$, where $w_n$ is the _n_ th word in the sequence.
- A _corpus_ is a collection of $M$ documents denoted by $D = \{\textbf{w}_1,\textbf{w}_2,... ,\textbf{w}_M \}$.

#### (2). Latent Dirichlet Allocation

LDA assumes the following priors and generating process for each document $\textbf{w}$ in a corpus D:
1. Choose $N \sim$ Poisson($\xi$).
2. Choose $\theta \sim$ Dir($\alpha$).
3. For each of the $N$ words $w_n$:
 - Choose a topic $z_n$ $\sim$ Multinomial($\theta$).
 - Choose a word $w_n$ from $p(w_n |z_n,\beta)$, a multinomial probability conditioned on the topic $z_n$.
where $N$ is the number of words and because it is independent of $\theta$ and $\textbf{z}$'s, its randomness could be ignored. $\theta$, representing the mixture of topics of a word, is the parameters for the distribution of $z_n$ with dimentionality $k$ which is assumed to be known and fixed. Utilizing the conjugacy of multinomial distribution and Dirichlet distribution, the inference process would be simpler. $\beta$ is a $k \times V$ matrix with $\beta_{ij} = p(w^j = 1 | z^i = 1)$.

By the definition and terminology, the likelihood of a document after integrating out the topics and mixture of topics is 

$$p(\textbf{w}|\alpha,\beta) = \int p(\theta|\alpha) (\prod_{n=1}^N \sum_{z_n} p(z_n|\theta) p(w_n|z_n,\beta)) d\theta$$

By multiplying over the corpus, the probability of a corpus is 

$$p(D|\alpha,\beta) = \prod_{d=1}^M \int p(\theta_d|\alpha) (\prod_{n=1}^{N_d} \sum_{z_{dn}} p(z_{dn}|\theta_d) p(w_{dn}|z_{dn},\beta)) d\theta_d$$

From the expression, we notice the summation within the multiplication, which causes the difficulty of inference directly. Therefore, the authors provide a method to infer the critical parameters by convexity-based variational inference and EM algorithm.

### B. Techniques for Inference

#### (1). Variational Inference

As the process suggested, $\beta$ and $z$ generating $w$ and $\theta$ generating $z$, the coupling between $\theta$ and $\beta$ could complicate the inference procedure. To address this problem, the authors modified the model and applied the variational inference to free $\beta$ and $\theta$ as $\gamma$ and $\phi_n$. This modification leads the model as 

$$q(\theta,\textbf{z}|\gamma,\phi) = q(\theta|\gamma) \prod_{n=1}^N q(z_n|\phi_n)$$

To determine the values of the variational parameters $\gamma$ and $\phi$, the authors decide to take the $(\gamma^*, \phi^*)$ that minimize the Kullback-Leibler (KL) divergence between the variational distribution and the true posterior distribution $p(\theta,\textbf{z}|\textbf{w},\alpha,\beta)$. The iterative update equations are

$$\phi_{ni} \propto \beta_{iw_n} \exp\{E_q[\log(\theta_i)|\gamma]\}$$

$$\gamma_i = \alpha_i + \sum_{n=1}^N \phi_{ni}$$

where 

$$E_q[\log(\theta_i)|\gamma] = \Psi(\gamma_i) - \Psi(\sum_{j=1}^k \gamma_j)$$

#### (2). EM Algorithm

At first, the intractable log likelihood for $\alpha$ and $\beta$ of the data is 

$$\ell(\alpha,\beta) = \sum_{d=1}^M \log p(\textbf{w}_d|\alpha,\beta)$$

By variational inference, the authors provide an alternative variational EM algorithm for the approximate empirical Bayes estimates for LDA model as finding the a lower bound with respect to the variational parameters $\gamma$ and $\phi$. The derivation of the variational EM algorithm for LDA yields the following iterative algorithm:
  1. (E-step) For each document, find the optimizing values of the variational parameters $\{\gamma^*_d , \phi^*_d : d \in D\}$. This is done as described in the previous section.
  2. (M-step) Maximize the resulting lower bound on the log likelihood with respect to the model parameters $\alpha$ and $\beta$. This corresponds to finding maximum likelihood estimates with expected sufficient statistics for each document under the approximate posterior which is computed in the E-step.
  3. Repeat all these two steps until the lower bound on the log likelihood converges.

By this procedure, the estimate of $\beta$ is

$$\beta_{ij} \propto \sum_{d=1}^M \sum_{n=1}^{N_d} \phi^*_{dni} w^j_{dn}$$

For the M-step of $\alpha$, the authors implement using a Newton-Raphson method.

## 3. Implementation on Real Dataset

In [55]:
import numpy as np
import scipy as sp
import pandas as pd
import random
import numba
from numba import jit, njit

In [56]:
f= open('voc', 'r')
voc = f.read().strip().split("\n")

In [57]:
def parse_vocabulary(vocab):
    """Parse the vocabulary set into two dictionaries, word with its index and index with its
word. When input is null, output should be null."""
    type_to_index = {}
    index_to_type = {}
    for word in set(vocab):
        index_to_type[len(index_to_type)] = word
        type_to_index[word] = len(type_to_index)        
    return (type_to_index, index_to_type)

In [58]:
# true data
type_to_index = parse_vocabulary(voc)[0]
index_to_type = parse_vocabulary(voc)[1]

In [59]:
ff = open('train', 'r')
train = ff.read().strip().split("\n")
fff = open('test', 'r')
test = fff.read().strip().split("\n")

In [60]:
def parse_data(corpus):
    """Parse the corpus into a list of documents. Find out the index of each word in each document and count its number of appearance in each document. When input is null, output should be null."""
    word_ids = []
    word_cts = []     
    for document_line in corpus:
        document_word_dict = {}
        for token in document_line.split():
            if token not in type_to_index:
                continue                
            type_id = type_to_index[token]
            if type_id not in document_word_dict:
                document_word_dict[type_id] = 0
            document_word_dict[type_id] += 1

        word_ids.append(np.array(list(document_word_dict.keys())))
        word_cts.append(np.array(list(document_word_dict.values()))[np.newaxis, :])

    return (word_ids, word_cts)

In [61]:
def compute_dirichlet_expectation(dirichlet_parameter):
    """Calculate the expectation of dirichlet parameter. When input is null, output should be a warning that the input is null."""
    if not np.array(dirichlet_parameter).size:
        return ("The dirichlet_parameter is null.")
    if (len(dirichlet_parameter.shape) == 1):
        return (sp.special.psi(dirichlet_parameter)-sp.special.psi(np.sum(dirichlet_parameter)))
    return (sp.special.psi(dirichlet_parameter) - sp.special.psi(np.sum(dirichlet_parameter, 1))[:, np.newaxis])

### Tests of function parse_vocabulary, parse_data, and compute_dirichlet_expectation, for both common and edge cases. Test results are consistent with the expectations which are described in docstrings.###

In [62]:
voc_test = ["year","state","new","percent","peopl","report","million","govern","presid"]
parse_vocabulary(voc_test)

({'govern': 4,
  'million': 8,
  'new': 1,
  'peopl': 7,
  'percent': 3,
  'presid': 6,
  'report': 2,
  'state': 5,
  'year': 0},
 {0: 'year',
  1: 'new',
  2: 'report',
  3: 'percent',
  4: 'govern',
  5: 'state',
  6: 'presid',
  7: 'peopl',
  8: 'million'})

In [63]:
voc_test = []
parse_vocabulary(voc_test)

({}, {})

In [64]:
corpus_test = ["mauric adult adult peopl year h last year resolv polic polic polic polic polic polic polic motiv appear cri christian christian three troubl pride farley farley friday lock bureau gun gun gun church marino marino","work posit reach reach peopl year thought million million million presid govern year year first percent state appear abl time billion told resid complet day develop made made made made wednesday wednesday","work offer shoot thought million far year year polic polic polic polic polic polic polic polic polic cash appear abl sophist told told woman woman arrest retain found day monday citi administr prepar fame art gun"]
parse_data(corpus_test)

([array([5569, 2372, 6262, 1286, 4039, 3042, 4742,  141,  656, 1427, 4438,
         3014, 5404,  416, 4258, 1253,  551, 6058, 6514, 4790, 5112, 4089]),
  array([5569, 1604,  709, 6793, 4749, 2388, 6486, 4089, 2008, 1115, 6306,
         5987, 1253,  296, 5293, 2545, 4786, 3829, 3637, 3065, 2299, 5694,
         1983]),
  array([5569, 4039, 2504, 1866,  845, 4942, 1176, 2019, 4089, 2008, 3176,
         1184, 6306, 5987, 4069, 4689, 5544, 2927, 2545,  882,  245, 6262,
         3065, 4865, 5694])],
 [array([[1, 2, 3, 1, 7, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2]]),
  array([[1, 1, 1, 1, 1, 2, 1, 3, 1, 2, 1, 3, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1]]),
  array([[1, 9, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1]])])

In [65]:
corpus_test = []
parse_data(corpus_test)

([], [])

In [66]:
number_of_topics = 100
number_of_types = len(type_to_index)
eta = np.random.gamma(100., 1./100., (number_of_topics,number_of_types))
compute_dirichlet_expectation(eta)

array([[-9.09684835, -9.55469276, -9.38363974, ..., -9.24216926,
        -9.35722427, -9.63080596],
       [-9.34649163, -9.30944856, -9.50675363, ..., -9.39547571,
        -9.40449805, -9.57647018],
       [-9.47861401, -9.74083413, -9.25161975, ..., -9.41531582,
        -9.48331717, -9.50022375],
       ..., 
       [-9.18447372, -9.42226028, -9.27308997, ..., -9.21091217,
        -9.10052444, -9.49658774],
       [-9.2591553 , -9.46987322, -8.98963486, ..., -9.43634688,
        -9.35462064, -9.37907071],
       [-9.45412259, -9.43736856, -9.48812175, ..., -9.36615048,
        -9.46575878, -9.393299  ]])

In [67]:
eta_test = []
compute_dirichlet_expectation(eta_test)

'The dirichlet_parameter is null.'

In [68]:
# Initialization
parsed_corpus = parse_data(train)
test_corpus = parse_data(test)
number_of_documents = len(parsed_corpus[0])

alpha_alpha = np.zeros(number_of_topics) + 1/number_of_topics
alpha_beta = np.zeros(number_of_types) + 1/number_of_types
gamma = np.zeros((number_of_documents, number_of_topics)) + alpha_alpha[np.newaxis, :] + 1.0 * number_of_types / number_of_topics       

hyper_parameter_optimize_interval=1

In [69]:
def e_step(corpus=None,local_parameter_iteration=50,number_of_topics=number_of_topics,
local_parameter_converge_threshold=1e-6):
    """E step. When input is None, output should be document_log_likelihood, phi_sufficient_statistics, and gamma. Otherwise, it should be words_log_likelihood, gamma_values."""
    if corpus == None:
        word_ids = parsed_corpus[0]
        word_cts = parsed_corpus[1]
    else:
        word_ids = corpus[0]
        word_cts = corpus[1]
    # Initialization 
    number_of_documents = len(word_ids)
    document_log_likelihood = 0
    words_log_likelihood = 0
    phi_sufficient_statistics = np.zeros((number_of_topics, number_of_types))
    gamma_values = np.zeros((number_of_documents, number_of_topics)) + alpha_alpha[np.newaxis, :] + 1.0 * number_of_types / number_of_topics
    E_log_eta = compute_dirichlet_expectation(eta)
    if parsed_corpus != None:
        E_log_prob_eta = E_log_eta - sp.misc.logsumexp(E_log_eta, axis=1)[:, np.newaxis]

    # iterate over all documents
    for doc_id in np.random.permutation(number_of_documents):
        # compute the total number of words
        total_word_count = np.sum(word_cts[doc_id])
        # initialize gamma for this document
        gamma_values[doc_id, :] = alpha_alpha + 1.0 * total_word_count / number_of_topics

        term_ids = word_ids[doc_id]
        term_counts = word_cts[doc_id]

        # update phi and gamma until gamma converges
        for gamma_iteration in range(local_parameter_iteration):
            log_phi = E_log_eta[:, term_ids].T + np.tile(sp.special.psi(gamma_values[doc_id, :]), (word_ids[doc_id].shape[0], 1))
            log_phi -= sp.misc.logsumexp(log_phi, axis=1)[:, np.newaxis]
            gamma_update = alpha_alpha + np.array(np.sum(np.exp(log_phi + np.log(np.repeat(term_counts, number_of_topics, axis=0).T)), axis=0))
            mean_change = np.mean(abs(gamma_update - gamma_values[doc_id, :]))
            gamma_values[doc_id, :] = gamma_update
            if mean_change <= local_parameter_converge_threshold:
                break

        # compute the alpha, gamma, and phi terms
        document_log_likelihood += sp.special.gammaln(np.sum(alpha_alpha)) - np.sum(sp.special.gammaln(alpha_alpha))
        document_log_likelihood += np.sum(sp.special.gammaln(gamma_values[doc_id, :])) - sp.special.gammaln(np.sum(gamma_values[doc_id, :]))
        document_log_likelihood -= np.sum(np.dot(term_counts, np.exp(log_phi) * log_phi))

# compute the p(w_{dn} | z_{dn}, \eta) terms, which will be cancelled during M-step
        words_log_likelihood += np.sum(np.exp(log_phi.T + np.log(term_counts)) * E_log_prob_eta[:, term_ids])      
        phi_sufficient_statistics[:, term_ids] += np.exp(log_phi + np.log(term_counts.transpose())).T
        
    if corpus == None:
        gamma = gamma_values
        return (document_log_likelihood, phi_sufficient_statistics, gamma)
    else:
        return (words_log_likelihood, gamma_values)

In [70]:
def m_step(phi_sufficient_statistics):
    """M step. When input is null, output should be a warning that the input is null."""
    if not np.array(phi_sufficient_statistics).size:
        return ("The input is null.")
    # compute the beta and the eta terms
    topic_log_likelihood = number_of_topics * (sp.special.gammaln(np.sum(alpha_beta)) - np.sum(sp.special.gammaln(alpha_beta)))
    topic_log_likelihood += np.sum(np.sum(sp.special.gammaln(eta), axis=1) - sp.special.gammaln(np.sum(eta, axis=1)))

    eta_temp = phi_sufficient_statistics + alpha_beta

    # compute the sufficient statistics for alpha and update
    alpha_sufficient_statistics = sp.special.psi(gamma) - sp.special.psi(np.sum(gamma, axis=1)[:, np.newaxis])
    alpha_sufficient_statistics = np.sum(alpha_sufficient_statistics, axis=0)  

    return (topic_log_likelihood, alpha_sufficient_statistics, eta_temp)

### Compute likelihood and sufficient statistics. Also serve as a testing when input of e_step is null. ###

In [71]:
document_log_likelihood, phi_sufficient_statistics, gamma = e_step()

### Implement M step to fit the model and test function m_step when input is null.###

In [72]:
topic_log_likelihood, alpha_sufficient_statistics, eta = m_step(phi_sufficient_statistics)
phi_sufficient_statistics_test = []
m_step(phi_sufficient_statistics_test)

'The input is null.'

In [73]:
def optimize_hyperparameters(alpha_sufficient_statistics, hyper_parameter_iteration=100, hyper_parameter_decay_factor=0.9, hyper_parameter_maximum_decay=10,alpha=alpha_alpha, hyper_parameter_converge_threshold=1e-6):
    """Optimize hyperparameter alpha. Since the function's input is the result of m_step function, it is unnecessary and difficult to test it. Just use it."""
    alpha_update = alpha        
    decay = 0
    for alpha_iteration in range(hyper_parameter_iteration):
        alpha_gradient = number_of_documents * (sp.special.psi(np.sum(alpha)) - sp.special.psi(alpha)) + alpha_sufficient_statistics
        alpha_hessian = -number_of_documents * sp.special.polygamma(1,alpha)

        sum_g_h = np.sum(alpha_gradient / alpha_hessian)
        sum_1_h = 1.0 / alpha_hessian
        z = number_of_documents * sp.special.polygamma(1, np.sum(alpha))
        c = sum_g_h / (1.0 / z + sum_1_h)

        # update the alpha vector
        while True:
            singular_hessian = False
            step_size = np.power(hyper_parameter_decay_factor, decay) * (alpha_gradient - c) / alpha_hessian               
            if np.any(alpha <= step_size):
                singular_hessian = True
            else: alpha_update = alpha - step_size

            if singular_hessian:
                decay += 1
                if decay > hyper_parameter_maximum_decay:
                    break
            else: break

        # check the alpha converge criteria
        mean_change = np.mean(abs(alpha_update - alpha))
        alpha = alpha_update
        if mean_change <= hyper_parameter_converge_threshold:
            break

    return (alpha)

In [74]:
alpha_alpha = optimize_hyperparameters(alpha_sufficient_statistics)
words_log_likelihood, corpus_gamma_values = e_step(corpus = test_corpus)

### For every topic, find out the top 20 words that are most likely to appear in this topic. ###

In [75]:
E_log_eta = compute_dirichlet_expectation(eta)
topic = []
for topic_index in range(number_of_topics):
    temp_list = []
    beta_probability = np.exp(E_log_eta[topic_index, :] - sp.misc.logsumexp(E_log_eta[topic_index, :]))
    i = 0
    for type_index in reversed(np.argsort(beta_probability)):
        i += 1
        if i <= 20:
            temp_list.append(index_to_type[type_index])
        else: break
    topic.append(temp_list)

index = random.sample(range(number_of_topics),10)
df = pd.DataFrame({"topic0":topic[index[9]],"topic1":topic[index[0]],"topic2":topic[index[1]],"topic3":topic[index[2]],"topic4":topic[index[3]],"topic5":topic[index[4]],"topic6":topic[index[5]],"topic7":topic[index[6]],"topic8":topic[index[7]],"topic9":topic[index[8]]})
df

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9
0,jackson,state,democrat,report,polic,soviet,japan,jackson,percent,new
1,support,parti,dukaki,militari,two,offici,govern,dukaki,report,year
2,robb,peopl,jackson,peopl,state,say,trade,parti,million,compani
3,campaign,say,offici,year,peopl,militari,new,vote,last,defen
4,democrat,govern,day,parti,offici,reagan,offici,presid,peopl,american
5,say,presid,aid,govern,new,presid,last,bush,state,peopl
6,dukaki,new,committ,soviet,charg,unit,report,democrat,compani,last
7,deleg,year,bush,nation,year,govern,two,two,dukaki,depart
8,presid,polic,first,forc,told,state,presid,state,year,soviet
9,convent,time,group,gorbachev,tuesday,union,week,nation,nation,system


## 4. Optimization

In [76]:
@jit
def compute_dirichlet_expectation(dirichlet_parameter):
    """Calculate the expectation of dirichlet parameter. When input is null, output should be a warning that the input is null."""
    if not np.array(dirichlet_parameter).size:
        return ("The dirichlet_parameter is null.")
    if (len(dirichlet_parameter.shape) == 1):
        return (sp.special.psi(dirichlet_parameter)-sp.special.psi(np.sum(dirichlet_parameter)))
    return (sp.special.psi(dirichlet_parameter) - sp.special.psi(np.sum(dirichlet_parameter, 1))[:, np.newaxis])


def e_step(corpus=None,local_parameter_iteration=50,number_of_topics=number_of_topics,
local_parameter_converge_threshold=1e-6):
    """E step. When input is None, output should be document_log_likelihood, phi_sufficient_statistics, and gamma. Otherwise, it should be words_log_likelihood, gamma_values."""
    if corpus == None:
        word_ids = parsed_corpus[0]
        word_cts = parsed_corpus[1]
    else:
        word_ids = corpus[0]
        word_cts = corpus[1]
    # Initialization 
    number_of_documents = len(word_ids)
    document_log_likelihood = 0
    words_log_likelihood = 0
    phi_sufficient_statistics = np.zeros((number_of_topics, number_of_types))
    gamma_values = np.zeros((number_of_documents, number_of_topics)) + alpha_alpha[np.newaxis, :] + 1.0 * number_of_types / number_of_topics
    E_log_eta = compute_dirichlet_expectation(eta)
    if parsed_corpus != None:
        E_log_prob_eta = E_log_eta - sp.misc.logsumexp(E_log_eta, axis=1)[:, np.newaxis]

    # iterate over all documents
    for doc_id in np.random.permutation(number_of_documents):
        # compute the total number of words
        total_word_count = np.sum(word_cts[doc_id])
        # initialize gamma for this document
        gamma_values[doc_id, :] = alpha_alpha + 1.0 * total_word_count / number_of_topics

        term_ids = word_ids[doc_id]
        term_counts = word_cts[doc_id]

        # update phi and gamma until gamma converges
        for gamma_iteration in range(local_parameter_iteration):
            log_phi = E_log_eta[:, term_ids].T + np.tile(sp.special.psi(gamma_values[doc_id, :]), (word_ids[doc_id].shape[0], 1))
            log_phi -= sp.misc.logsumexp(log_phi, axis=1)[:, np.newaxis]
            gamma_update = alpha_alpha + np.array(np.sum(np.exp(log_phi + np.log(np.repeat(term_counts, number_of_topics, axis=0).T)), axis=0))
            mean_change = np.mean(abs(gamma_update - gamma_values[doc_id, :]))
            gamma_values[doc_id, :] = gamma_update
            if mean_change <= local_parameter_converge_threshold:
                break

        # compute the alpha, gamma, and phi terms
        document_log_likelihood += sp.special.gammaln(np.sum(alpha_alpha)) - np.sum(sp.special.gammaln(alpha_alpha))
        document_log_likelihood += np.sum(sp.special.gammaln(gamma_values[doc_id, :])) - sp.special.gammaln(np.sum(gamma_values[doc_id, :]))
        document_log_likelihood -= np.sum(np.dot(term_counts, np.exp(log_phi) * log_phi))

# compute the p(w_{dn} | z_{dn}, \eta) terms, which will be cancelled during M-step
        words_log_likelihood += np.sum(np.exp(log_phi.T + np.log(term_counts)) * E_log_prob_eta[:, term_ids])      
        phi_sufficient_statistics[:, term_ids] += np.exp(log_phi + np.log(term_counts.transpose())).T
        
    if corpus == None:
        gamma = gamma_values
        return (document_log_likelihood, phi_sufficient_statistics, gamma)
    else:
        return (words_log_likelihood, gamma_values)

In [77]:
%%time
document_log_likelihood, phi_sufficient_statistics, gamma = e_step()

CPU times: user 53.7 s, sys: 548 ms, total: 54.3 s
Wall time: 54.6 s


In [78]:
def compute_dirichlet_expectation(dirichlet_parameter):
    """Calculate the expectation of dirichlet parameter. When input is null, output should be a warning that the input is null."""
    if not np.array(dirichlet_parameter).size:
        return ("The dirichlet_parameter is null.")
    if (len(dirichlet_parameter.shape) == 1):
        return (sp.special.psi(dirichlet_parameter)-sp.special.psi(np.sum(dirichlet_parameter)))
    return (sp.special.psi(dirichlet_parameter) - sp.special.psi(np.sum(dirichlet_parameter, 1))[:, np.newaxis])


def e_step(corpus=None,local_parameter_iteration=50,number_of_topics=number_of_topics,
local_parameter_converge_threshold=1e-6):
    """E step. When input is None, output should be document_log_likelihood, phi_sufficient_statistics, and gamma. Otherwise, it should be words_log_likelihood, gamma_values."""
    if corpus == None:
        word_ids = parsed_corpus[0]
        word_cts = parsed_corpus[1]
    else:
        word_ids = corpus[0]
        word_cts = corpus[1]
    # Initialization 
    number_of_documents = len(word_ids)
    document_log_likelihood = 0
    words_log_likelihood = 0
    phi_sufficient_statistics = np.zeros((number_of_topics, number_of_types))
    gamma_values = np.zeros((number_of_documents, number_of_topics)) + alpha_alpha[np.newaxis, :] + 1.0 * number_of_types / number_of_topics
    E_log_eta = compute_dirichlet_expectation(eta)
    if parsed_corpus != None:
        E_log_prob_eta = E_log_eta - sp.misc.logsumexp(E_log_eta, axis=1)[:, np.newaxis]

    # iterate over all documents
    for doc_id in np.random.permutation(number_of_documents):
        # compute the total number of words
        total_word_count = np.sum(word_cts[doc_id])
        # initialize gamma for this document
        gamma_values[doc_id, :] = alpha_alpha + 1.0 * total_word_count / number_of_topics

        term_ids = word_ids[doc_id]
        term_counts = word_cts[doc_id]

        # update phi and gamma until gamma converges
        for gamma_iteration in range(local_parameter_iteration):
            log_phi = E_log_eta[:, term_ids].T + np.tile(sp.special.psi(gamma_values[doc_id, :]), (word_ids[doc_id].shape[0], 1))
            log_phi -= sp.misc.logsumexp(log_phi, axis=1)[:, np.newaxis]
            gamma_update = alpha_alpha + np.array(np.sum(np.exp(log_phi + np.log(np.repeat(term_counts, number_of_topics, axis=0).T)), axis=0))
            mean_change = np.mean(abs(gamma_update - gamma_values[doc_id, :]))
            gamma_values[doc_id, :] = gamma_update
            if mean_change <= local_parameter_converge_threshold:
                break

        # compute the alpha, gamma, and phi terms
        document_log_likelihood += sp.special.gammaln(np.sum(alpha_alpha)) - np.sum(sp.special.gammaln(alpha_alpha))
        document_log_likelihood += np.sum(sp.special.gammaln(gamma_values[doc_id, :])) - sp.special.gammaln(np.sum(gamma_values[doc_id, :]))
        document_log_likelihood -= np.sum(np.dot(term_counts, np.exp(log_phi) * log_phi))

# compute the p(w_{dn} | z_{dn}, \eta) terms, which will be cancelled during M-step
        words_log_likelihood += np.sum(np.exp(log_phi.T + np.log(term_counts)) * E_log_prob_eta[:, term_ids])      
        phi_sufficient_statistics[:, term_ids] += np.exp(log_phi + np.log(term_counts.transpose())).T
        
    if corpus == None:
        gamma = gamma_values
        return (document_log_likelihood, phi_sufficient_statistics, gamma)
    else:
        return (words_log_likelihood, gamma_values)

In [79]:
%%time
document_log_likelihood, phi_sufficient_statistics, gamma = e_step()

CPU times: user 52.8 s, sys: 437 ms, total: 53.2 s
Wall time: 53.4 s


## 5. Discussion and analysis

### Change the number of topics and compare the results.###

**The reason why the following code is not wrapped into a function is that many global variables are used and there will be conflicts if putting it in a function.**

In [80]:
number_of_topics = 10
alpha_alpha = np.zeros(number_of_topics) + 1/number_of_topics
alpha_beta = np.zeros(number_of_types) + 1/number_of_types
eta = np.random.gamma(100., 1./100., (number_of_topics,number_of_types))
gamma = np.zeros((number_of_documents, number_of_topics)) + alpha_alpha[np.newaxis, :] + 1.0 * number_of_types / number_of_topics 
document_log_likelihood, phi_sufficient_statistics, gamma = e_step(number_of_topics=number_of_topics)
topic_log_likelihood, alpha_sufficient_statistics, eta = m_step(phi_sufficient_statistics)
alpha_alpha = optimize_hyperparameters(alpha_sufficient_statistics,alpha=alpha_alpha)
words_log_likelihood, corpus_gamma_values = e_step(corpus = test_corpus,
number_of_topics=number_of_topics)
E_log_eta = compute_dirichlet_expectation(eta)
topic = []
for topic_index in range(number_of_topics):
    temp_list = []
    beta_probability = np.exp(E_log_eta[topic_index, :] - sp.misc.logsumexp(E_log_eta[topic_index, :]))
    i = 0
    for type_index in reversed(np.argsort(beta_probability)):
        i += 1
        if i <= 20:
            temp_list.append(index_to_type[type_index])
        else: break
    topic.append(temp_list)

index = random.sample(range(number_of_topics),10)
df = pd.DataFrame({"topic0":topic[index[9]],"topic1":topic[index[0]],"topic2":topic[index[1]],"topic3":topic[index[2]],"topic4":topic[index[3]],"topic5":topic[index[4]],"topic6":topic[index[5]],"topic7":topic[index[6]],"topic8":topic[index[7]],"topic9":topic[index[8]]})
df

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9
0,percent,year,state,year,year,year,percent,year,two,peopl
1,state,percent,new,govern,new,million,year,state,new,year
2,nation,say,year,state,soviet,percent,report,last,year,new
3,presid,state,report,compani,million,billion,new,two,presid,presid
4,peopl,million,peopl,offici,percent,new,say,offici,state,state
5,say,govern,million,presid,presid,peopl,day,say,peopl,bush
6,year,new,govern,two,polic,last,time,polic,nation,soviet
7,unit,report,cent,last,two,nation,soviet,nation,govern,say
8,new,hou,polic,peopl,state,state,govern,report,polic,report
9,report,peopl,nation,unit,report,month,offici,govern,offici,offici


**If we just have 10 topics, we can see that words in different topics are similar. Because test corpus is not that large and there are not so many key words, the corpus is not fully divided into different topics and different topics will overlap with each other. So when the number of topics is small, those key words will appear repeatedly and the distinctions of topics are not obvious.**